In [2]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"

# Load YOLO with the full paths
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load Video
cap = cv2.VideoCapture('traffic_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Prepare the frame for the YOLO model
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Process the detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Filter weak detections
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Max Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw the detections on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0) if label == "car" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow("Traffic Detection", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
with open("yolov3.cfg", "r") as f:
    print(f.read())


[net]
# Testing
# batch=1
# subdivisions=1
# Training
batch=64
subdivisions=16
width=608
height=608
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutiona

In [4]:
import os
print(os.getcwd())


d:\SIH


In [5]:
test_image_path = r"D:\SIH\traffic.jpg"
img = cv2.imread(test_image_path)
if img is None:
    print("Failed to load image.")


In [6]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the image
image_path = r"D:\SIH\traffic.jpg"  # Updated path to your image
img = cv2.imread(image_path)

# Check if image is loaded properly
if img is None:
    raise ValueError(f"Image at path {image_path} could not be loaded.")

height, width, channels = img.shape

# Prepare the image for the neural network
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Initialize lists to hold the detection data
class_ids = []
confidences = []
boxes = []

# Parse the network's output
for out in outs:
    for detection in out:
        if len(detection) > 5:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Filter out weak detections
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate the coordinates of the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw bounding boxes on the image
# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw bounding boxes on the image
for i in range(len(boxes)):
    if i in indices:
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = str(class_ids[i])
        confidence = confidences[i]
        color = (0, 255, 0)  # Green color for bounding boxes
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [7]:
# Print the shapes of the outputs
for out in outs:
    print(out.shape)


(507, 85)
(2028, 85)
(8112, 85)


In [31]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the image
image_path = r"D:\SIH\traffic.jpg"  # Updated path to your image
img = cv2.imread(image_path)

# Check if image is loaded properly
if img is None:
    raise ValueError(f"Image at path {image_path} could not be loaded.")

height, width, channels = img.shape

# Prepare the image for the neural network
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Print output shapes for debugging
for i, out in enumerate(outs):
    print(f"Output {i} shape: {out.shape}")

# Initialize lists to hold the detection data
class_ids = []
confidences = []
boxes = []

# Parse the network's output
for out in outs:
    for detection in out:
        print(f"Detection type: {type(detection)}, shape: {detection.shape}")
        for obj in detection:
            # Ensure obj is an array and has the expected number of elements
            if isinstance(obj, np.ndarray) and len(obj) > 5:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:  # Filter out weak detections
                    center_x = int(obj[0] * width)
                    center_y = int(obj[1] * height)
                    w = int(obj[2] * width)
                    h = int(obj[3] * height)

                    # Calculate the coordinates of the bounding box
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw bounding boxes on the image
if len(indices) > 0:
    indices = indices.flatten()  # Flatten the array of indices
    for i in indices:
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = str(class_ids[i])
        confidence = confidences[i]
        color = (0, 255, 0)  # Green color for bounding boxes
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display the image with detections
cv2.imshow("Image", img)
cv2.waitKey(2000)  # Wait for 2 seconds before closing the window
cv2.destroyAllWindows()



Output 0 shape: (507, 85)
Output 1 shape: (2028, 85)
Output 2 shape: (8112, 85)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
Detection type: <class 'numpy.ndarray'>, shape: (85,)
De

In [16]:
# Display the image with detections
cv2.imshow("Image", img)
cv2.waitKey(2000)  # Wait for 2 seconds before closing the window
cv2.destroyAllWindows()

In [15]:
import cv2

# Test displaying a simple image
img = cv2.imread(r"D:\SIH\traffic.jpg")
if img is None:
    print("Failed to load image.")
else:
    cv2.imshow("Test Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [32]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the image
image_path = r"D:\SIH\traffic.jpg"  # Updated path to your image
img = cv2.imread(image_path)

# Check if image is loaded properly
if img is None:
    raise ValueError(f"Image at path {image_path} could not be loaded.")

height, width, channels = img.shape

# Prepare the image for the neural network
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Initialize lists to hold the detection data
class_ids = []
confidences = []
boxes = []

# Parse the network's output
for out in outs:
    for detection in out:
        scores = detection[5:]  # Get the scores for the detected classes
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Filter out weak detections
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Calculate the coordinates of the bounding box
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Check if indices is empty
if len(indices) == 0:
    print("No objects detected.")
else:
    # Handle different cases for `indices`
    if isinstance(indices, tuple):
        indices = indices[0]

    # Ensure indices is a numpy array
    if isinstance(indices, np.ndarray):
        indices = indices.flatten()

    # Draw bounding boxes on the image
    for i in indices:
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = str(class_ids[i])
        confidence = confidences[i]
        color = (0, 255, 0)  # Green color for bounding boxes
        thickness = 2  # Thickness of the bounding box lines
        cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness)
        cv2.putText(img, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

# Save the image with detections to check results
output_path = r"D:\SIH\output_image.jpg"
cv2.imwrite(output_path, img)
print(f"Image saved to {output_path}")

# Display the image with detections
cv2.imshow("Image", img)
cv2.waitKey(20000)  # Wait for 2 seconds before closing the window
cv2.destroyAllWindows()

Image saved to D:\SIH\output_image.jpg


In [29]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"
classes_file = r"D:\SIH\classes.names"  # Path to the class labels file

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Path to the class labels file
classes_file = r"D:\SIH\classes.names"  # Make sure this path is correct

# Load class labels
with open(classes_file, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]


# Open the video file
video_path = r"D:\SIH\traffic_vid.mp4"  # Path to your video file
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    raise ValueError(f"Video file at path {video_path} could not be opened.")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object to save output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(r"D:\SIH\traffic_output.avi", fourcc, fps, (width, height))

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Break the loop if no frame is read
    if not ret:
        break
    
    # Prepare the frame for the neural network
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists to hold the detection data
    class_ids = []
    confidences = []
    boxes = []

    # Parse the network's output
    for out in outs:
        for detection in out:
            for obj in detection:
                if isinstance(obj, np.ndarray) and len(obj) > 5:
                    scores = obj[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(obj[0] * width)
                        center_y = int(obj[1] * height)
                        w = int(obj[2] * width)
                        h = int(obj[3] * height)

                        # Calculate the coordinates of the bounding box
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

    # Apply non-maxima suppression to remove overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes on the frame
    for i in indices.flatten():
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = class_names[class_ids[i]]
        confidence = confidences[i]
        color = (0, 255, 0)  # Green color for bounding boxes
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the frame with detections to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2.imshow("Video", frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\SIH\\classes.names'

In [30]:
import os

# Path to the class labels file
classes_file = r"D:\SIH\classes.names"

# Check if the file exists
if os.path.isfile(classes_file):
    print("File exists.")
else:
    print("File does not exist.")


File does not exist.


In [28]:
classes_file = r"D:\SIH\classes.names"


In [1]:
import cv2
import numpy as np

# Specify the full paths to the weight and config files
weights_path = r"D:\SIH\yolov3.weights"
config_path = r"D:\SIH\yolov3.cfg"
classes_file = r"D:\SIH\coco.names"  # Path to the class labels file

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels
with open(classes_file, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

# Open the video file
video_path = r"D:\SIH\traffic_vid.mp4"  # Path to your video file
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    raise ValueError(f"Video file at path {video_path} could not be opened.")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object to save output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(r"D:\SIH\traffic_output.mp4", fourcc, fps, (width, height))

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Break the loop if no frame is read
    if not ret:
        break
    
    # Prepare the frame for the neural network
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists to hold the detection data
    class_ids = []
    confidences = []
    boxes = []

    # Parse the network's output
    for out in outs:
        for detection in out:
            for obj in detection:
                if isinstance(obj, np.ndarray) and len(obj) > 5:
                    scores = obj[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(obj[0] * width)
                        center_y = int(obj[1] * height)
                        w = int(obj[2] * width)
                        h = int(obj[3] * height)

                        # Calculate the coordinates of the bounding box
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

    # Apply non-maxima suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Check if indices is a tuple and extract the correct element if needed
if isinstance(indices, tuple):
    indices = indices[0]

# Draw bounding boxes on the frame
for i in indices.flatten():
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    label = class_names[class_ids[i]]
    confidence = confidences[i]
    color = (0, 255, 0)  # Green color for bounding boxes
    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
    cv2.putText(frame, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


KeyboardInterrupt: 

In [2]:
import cv2
import numpy as np

# Load YOLO with CUDA support (if available)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load Video
cap = cv2.VideoCapture('traffic_vid_2.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller size for faster processing
    frame = cv2.resize(frame, (320, 320))  # Resize to 320x320 or 416x416 for faster performance
    height, width, channels = frame.shape

    # Prepare the frame for the YOLO model
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Process the detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Filter weak detections
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Max Suppression with optimized values
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)

    # Draw the detections on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0) if label == "car" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow("Traffic Detection", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

In [7]:
import cv2
import numpy as np

# Load YOLO with CUDA support (if available)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load Video
cap = cv2.VideoCapture('traffic_vid_2.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller size for faster processing
    frame = cv2.resize(frame, (320, 320))  # Resize to 320x320 or 416x416 for faster performance
    height, width, channels = frame.shape

    # Prepare the frame for the YOLO model
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Process the detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Filter weak detections
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Max Suppression with optimized values
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)

    # Draw the detections on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0) if label == "car" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow("Traffic Detection", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import time

# Load YOLO with a smaller model for faster performance
net = cv2.dnn.readNet("yolov4-tiny.weights", "yolov4-tiny.cfg")  # Use YOLOv4-tiny for faster detection
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load Video
cap = cv2.VideoCapture('traffic_vid_2.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Resize the frame for faster processing
    new_width, new_height = 416, 416
    frame_resized = cv2.resize(frame, (new_width, new_height))

    # Prepare the frame for the YOLO model
    blob = cv2.dnn.blobFromImage(frame_resized, 0.00392, (new_width, new_height), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    start_time = time.time()
    outs = net.forward(output_layers)
    end_time = time.time()
    print(f"Frame processing time: {end_time - start_time:.2f} seconds")

    class_ids = []
    confidences = []
    boxes = []

    # Process the detections
    for out in outs:
        for detection in out:
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:  # Filter weak detections
                    center_x = int(obj[0] * new_width)
                    center_y = int(obj[1] * new_height)
                    w = int(obj[2] * new_width)
                    h = int(obj[3] * new_height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    # Scale back up to original frame size
                    x = int(x * width / new_width)
                    y = int(y * height / new_height)
                    w = int(w * width / new_width)
                    h = int(h * height / new_height)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

    # Apply Non-Max Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw the detections on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0) if label == "car" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow("Traffic Detection", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()